In [3]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.keras.models import load_model
MODEL_PATH = "IV35_Enhanced3.h5"
model = load_model(MODEL_PATH)
IMG_SIZE = (224, 224)

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
def load_and_preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=IMG_SIZE)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Error loading or preprocessing image: {e}")
        return None

In [6]:
def predict_image(img):
    img_array = load_and_preprocess_image(img)
    if img_array is None:
        return "Error loading image"

    predictions = model.predict(img_array)
    gender_prob = predictions[0][0][0]
    age_probabilities = predictions[1][0]

    gender = "Male" if gender_prob <= 0.5 else "Female"
    age_category = np.argmax(age_probabilities)

    result = {
        'gender': gender,
        'gender_probability': float(gender_prob),
        'age_category': int(age_category),
        'age_range': f"{age_category*5}-{age_category*5 + 4}"
    }

    return result


In [7]:
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.components.Image(type="filepath"),
    outputs="json",
    title="Age and Gender Prediction",
    description="Upload an image to predict age and gender."
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 2s 2s/step
